In [ ]:
import pandas as pd
file_path = "/home/sagemaker-user/TrajCL/data/porto_20200"
trajs = pd.read_pickle(file_path)

In [ ]:
trajs.head()

In [ ]:
# from torch.utils.data import Dataset
# class TrajDataset(Dataset):
#     def __init__(self, data):
#         # data: DataFrame
#         self.data = data

#     def __getitem__(self, index):
#         return self.data.loc[index].merc_seq

#     def __len__(self):
#         return self.data.shape[0]

In [ ]:
# l = trajs.shape[0]
# train_idx = (int(l*0), 200000)
# eval_idx = (int(l*0.7), int(l*0.8))
# test_idx = (int(l*0.8), int(l*1.0))

# train = TrajDataset(trajs[train_idx[0]: train_idx[1]])
# eval = TrajDataset(trajs[eval_idx[0]: eval_idx[1]])
# test = TrajDataset(trajs[test_idx[0]: test_idx[1]])

In [ ]:
idx = 15
lon_lat_list = trajs['wgs_seq'][idx]

In [ ]:
lon_lat_list

In [ ]:
# !pip install folium
import folium

from folium import plugins
coordinates = [[x[0],x[1]] for x in lon_lat_list]
# Initialize map
m = folium.Map(location=coordinates[0], zoom_start=30)

folium.Marker(
    location=coordinates[0],
    popup="Start Location",
    icon=folium.Icon(color='green')  # Color can be 'red', 'blue', 'green', 'purple', etc.
).add_to(m)
# Add markers
for lat, lon in coordinates[1:]:
    folium.Marker([lat, lon]).add_to(m)

# Draw arrows between points
for i in range(len(coordinates) - 1):
    start = coordinates[i]
    end = coordinates[i + 1]

    # Draw the line
    line = folium.PolyLine([start, end], color="blue", weight=3, opacity=0.7).add_to(m)

    # Add directional arrow
    plugins.PolyLineTextPath(
        line,
        '➤',  # arrow symbol
        repeat=True,
        offset=7,
        attributes={'fill': 'blue', 'font-weight': 'bold', 'font-size': '16'}
    ).add_to(m)

# Save and show the map
m.save("map_with_arrows.html")
m


In [ ]:
import sys
sys.path.append("..")
from utils.cellspace import CellSpace

In [ ]:
import pickle

porto_cs = pickle.load(open("/home/sagemaker-user/TrajCL/data/porto_20200_cell100_cellspace.pkl","rb"))


In [ ]:
porto_cs

In [ ]:
porto_cs.size()

In [ ]:
lon_lat_list = [[-8.660646, 41.168574],
 [-8.661087, 41.167926],
 [-8.661231, 41.166576],
 [-8.660637, 41.166396],
 [-8.660295, 41.166819],
 [-8.658954, 41.168394],
 [-8.657649, 41.169906],
 [-8.656371, 41.171454],
 [-8.654706, 41.173479],
 [-8.653014, 41.17527],
 [-8.651349, 41.17644],
 [-8.652213, 41.177241],
 [-8.651358, 41.178069],
 [-8.65071, 41.178924],
 [-8.65125, 41.17968],
 [-8.649648, 41.180643],
 [-8.647515, 41.18193],
 [-8.644491, 41.183541],
 [-8.641701, 41.184108],
 [-8.638677, 41.182452],
 [-8.635284, 41.180589],
 [-8.632476, 41.179104],
 [-8.629983, 41.179365],
 [-8.629857, 41.179374],
 [-8.629857, 41.179374],
 [-8.629263, 41.179437],
 [-8.625996, 41.179779],
 [-8.622018, 41.180463],
 [-8.61885, 41.181912],
 [-8.617077, 41.182821],
 [-8.614215, 41.183082],
 [-8.613675, 41.183136],
 [-8.613702, 41.183145],
 [-8.613288, 41.18319],
 [-8.610813, 41.184],
 [-8.607474, 41.184648],
 [-8.604018, 41.183892],
 [-8.601768, 41.183253],
 [-8.601588, 41.182632],
 [-8.601957, 41.181831],
 [-8.601903, 41.181795],
 [-8.601948, 41.181822],
 [-8.601894, 41.181813]]

In [ ]:
porto_cs.x_min, porto_cs.x_max

In [ ]:
from utils.tool_funcs import lonlat2meters

In [ ]:
lonlat2meters(lon_lat_list[0][0],lon_lat_list[0][1])

In [ ]:
point_ids = [porto_cs.get_cellid_by_point(*lonlat2meters(x[0],x[1])) for x in lon_lat_list]

In [ ]:
point_ids

In [ ]:
from ah_databricks_data_loader import DatabricksDataLoader

# Instantiate the DatabricksDataLoader.
ddl = DatabricksDataLoader()

# List available shares.
shares = ddl.list_shares()

# Print shares.
print(shares)

In [ ]:
9069952

In [ ]:
from ah_databricks_data_loader import DatabricksDataLoader

# Instantiate the DatabricksDataLoader.
ddl = DatabricksDataLoader()

# Load data.
df = ddl.load_as_spark(schema="datascience_scratchpad", table="all_traj_val")

# Display the loaded DataFrame.
df.count()

In [ ]:
count=0
for row in df.toLocalIterator():
    print(row)
    count+=1
    if count>10:
        break

In [ ]:
row['traj_id']

In [ ]:
splits = df.randomSplit([0.8, 0.1, 0.1], seed=42)  # seed for reproducibility
train_df, val_df, test_df = splits

In [ ]:
train_df.show()

In [ ]:
df_with_index = df.rdd.zipWithIndex().map(lambda x: (x[1], x[0])).toDF(["row_index", "row"])

In [ ]:
df_with_index.filter(df_with_index['row_index']==2).collect()[0]

In [ ]:
next(df.toLocalIterator())

In [ ]:
df.repartition(10).write.parquet("/mnt/sagemaker-nvme/data/usa/val", mode="overwrite")

In [ ]:
import os
data_dir = "/home/sagemaker-user/TrajCL/data/nyc/train_with_ts/"
files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".parquet")]
files

import pandas as pd
from tqdm import tqdm
total_count = 0
for file_path in tqdm(files):
    df = pd.read_parquet(file_path)
    total_count+=len(df)

total_count

In [ ]:
test = pd.read_parquet("/home/sagemaker-user/TrajCL/data/parquet_files/nyc_v2/part-00000-76372e3b-95b4-498e-9e85-2765482fcef5-c000.snappy.parquet")
test.head()

In [ ]:
len(test)

In [ ]:
import numpy as np
np.stack(test['merc_seq_filtered'].values[0])

In [ ]:
import os
import torch
from torch.utils.data import IterableDataset
import pandas as pd
import numpy as np
class ParquetDataset(IterableDataset):
    def __init__(self, data_dir):
        self.files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".parquet")]

    def __iter__(self):
        for file_path in self.files:
            df = pd.read_parquet(file_path)
            features = df["merc_seq_filtered"].values
            for feature in features:
                yield torch.tensor(np.stack(feature))

In [ ]:
parquet_dataset = ParquetDataset("/home/sagemaker-user/TrajCL/data/parquet_files/nyc/")

In [ ]:
from torch.utils.data import DataLoader
dl = DataLoader(parquet_dataset, batch_size=1)

In [ ]:
count = 0
for batch in dl:
    print(batch)
    count+=1
    if count>10:
        break